## 解析XML文件
### 1. 获取XML文件树  
   `DecodeXML(file)`  
   输入：XML文件路径  
   输出：xml.dom的树状结构的根节点  
   `return DOMTreeRoot`  
     
### 2. 获取动画信息  
   `GetAnimationInfo(DOMTreeRoot)`  
   输入：xml.dom的树状结构的根节点  
   输出：包含动画文件信息的字典  
   
```PYTHON
return AnimationInfo = 
   {'Fps': '30', 'Version': '153', 'LayerInfo': 
      [{'Id': '0', 'Name': 'glow', 'Path': 'characters/costumes/Character_001_Isaac.png'}, 
      {'Id': '1', 'Name': 'body', 'Path': 'characters/costumes/Character_001_Isaac.png'}, 
      ...
      {'Id': '14', 'Name': 'back', 'Path': 'characters/costumes/Character_001_Isaac.png'}
   ]}
```
  
### 3. 获取动画帧信息  
   `GetAnimationFrame(DOMTreeRoot)`   
   输入：xml.dom的树状结构的根节点  
   输出：包含动画帧信息的列表。帧信息包含在帧中，所有帧包含在关键帧列表中，关键帧列表是图层的一个元素，图层包含在图层列表中，图层列表是动画的一个元素，动画包含在动画列表中。  
   
```PYTHON
return AnimaitionList = 
   [{'Name': 'Pickup', 'FrameNum': '42', 'LayerAnimationList': [
      {'LayerId': '0', 'Visible': 'true', 'FrameList': []}
      {'LayerId': '1', 'Visible': 'true', 'FrameList': []}
      ...
      {'LayerId': '12', 'Visible': 'true', 'FrameList': [
         {'XPosition': '0', 'YPosition': '0', 'XPivot': '32', 'YPivot': '56', 'XCrop': '0', ... , 'Interpolated': 'true'}
         {'XPosition': '0', 'YPosition': '2', 'XPivot': '32', 'YPivot': '56', 'XCrop': '64', ... , 'Interpolated': 'true'}
         ...
      ]}
      {'LayerId': '14', 'Visible': 'true', 'FrameList': []}
   ]}, 
   {'Name': 'Hit', 'FrameNum': '8', 'LayerAnimationList': []},
   ...
   {'Name': 'DeathTeleport', 'FrameNum': '21', 'LayerAnimationList': []}]
```

In [85]:
# 读取xml文件（其实是.anm2文件），获取动画的信息
from xml.dom.minidom import parse
import xml.dom.minidom

def DecodeXML(file):
    # 使用minidom解析器打开 XML 文档
    DOMTree = xml.dom.minidom.parse(file)
    # 获取文档唯一根节点
    DOMTreeRoot = DOMTree.documentElement
    return DOMTreeRoot

def GetAnimationInfo(DOMTreeRoot):
    AnimationInfo = {}
    XML_InfoList = DOMTreeRoot.getElementsByTagName("Info")
    for XML_Info in XML_InfoList:
        AnimationInfo["Fps"] = XML_Info.getAttribute("Fps")
        AnimationInfo["Version"] = XML_Info.getAttribute("Version")
    ##
    XML_SpritesheetList = DOMTreeRoot.getElementsByTagName("Spritesheet")
    SpritesheetDir = {}
    for XML_Spritesheet in XML_SpritesheetList:
        SpritesheetDir[XML_Spritesheet.getAttribute("Id")] = XML_Spritesheet.getAttribute("Path")
    # print(SpritesheetDir)
    ##
    XML_LayerList = DOMTreeRoot.getElementsByTagName("Layer")
    AnimationInfo["LayerInfo"] = []
    for XML_Layer in XML_LayerList:
        LayerInfoDir = {}
        LayerInfoDir["Id"] = XML_Layer.getAttribute("Id")
        LayerInfoDir["Name"] = XML_Layer.getAttribute("Name")
        LayerInfoDir["Path"] = SpritesheetDir[XML_Layer.getAttribute("SpritesheetId")]
        AnimationInfo["LayerInfo"].append(LayerInfoDir)
    # print(AnimationInfo)
    return AnimationInfo

def GetAnimationFrame(DOMTreeRoot):
    AnimaitionList =  []
    XML_AnimationList = DOMTreeRoot.getElementsByTagName("Animation")
    for XML_Animation in XML_AnimationList:
        AnimationDir = {}
        AnimationDir["Name"] = XML_Animation.getAttribute("Name")
        AnimationDir["FrameNum"] = int(XML_Animation.getAttribute("FrameNum"))
        AnimationDir["LayerAnimationList"] = []
        XML_LayerAnimationList = XML_Animation.getElementsByTagName("LayerAnimation")
        for XML_LayerAnimation in XML_LayerAnimationList:
            LayerAnimationDir = {}
            LayerAnimationDir["LayerId"] = XML_LayerAnimation.getAttribute("LayerId")
            LayerAnimationDir["Visible"] = XML_LayerAnimation.getAttribute("Visible")
            LayerAnimationDir["FrameList"] = []
            XML_FrameList = XML_LayerAnimation.getElementsByTagName("Frame")
            for XML_Frame in XML_FrameList:
                FrameDir = {}
                FrameDir['XPosition'] = int(XML_Frame.getAttribute('XPosition'))
                FrameDir['YPosition'] = int(XML_Frame.getAttribute('YPosition'))
                FrameDir['XPivot'] = int(XML_Frame.getAttribute('XPivot'))
                FrameDir['YPivot'] = int(XML_Frame.getAttribute('YPivot'))
                FrameDir['XCrop'] = int(XML_Frame.getAttribute('XCrop'))
                FrameDir['YCrop'] = int(XML_Frame.getAttribute('YCrop'))
                FrameDir['Width'] = int(XML_Frame.getAttribute('Width'))
                FrameDir['Height'] = int(XML_Frame.getAttribute('Height'))
                FrameDir['XScale'] = int(XML_Frame.getAttribute('XScale'))
                FrameDir['YScale'] = int(XML_Frame.getAttribute('YScale'))
                FrameDir['Delay'] = int(XML_Frame.getAttribute('Delay'))
                FrameDir['Visible'] = XML_Frame.getAttribute('Visible')
                FrameDir['RedTint'] = int(XML_Frame.getAttribute('RedTint'))
                FrameDir['GreenTint'] = int(XML_Frame.getAttribute('GreenTint'))
                FrameDir['BlueTint'] = int(XML_Frame.getAttribute('BlueTint'))
                FrameDir['AlphaTint'] = int(XML_Frame.getAttribute('AlphaTint'))
                FrameDir['RedOffset'] = int(XML_Frame.getAttribute('RedOffset'))
                FrameDir['GreenOffset'] = int(XML_Frame.getAttribute('GreenOffset'))
                FrameDir['BlueOffset'] = int(XML_Frame.getAttribute('BlueOffset'))
                FrameDir['Rotation'] = int(XML_Frame.getAttribute('Rotation'))
                FrameDir['Interpolated'] = XML_Frame.getAttribute('Interpolated')
                LayerAnimationDir["FrameList"].append(FrameDir)
            AnimationDir["LayerAnimationList"].append(LayerAnimationDir)
        AnimaitionList.append(AnimationDir)
    return AnimaitionList
    


## 插入过度帧
FrameList中只保存了关键帧信息，如果Frame中Delay的值大于1，说明需要插入过度帧  
如果Interpolated属性为false，则只需要将关键帧的内容复制到过度帧中  
如果Interpolated属性为true，则需要根据下一个关键帧中某些属性的值计算过度帧中的属性值。  
其中某些属性不进行插值处理，如Position、Scale、Tint、ColorOffset、colorRotation等值，直接复制关键帧中的值  
某些图层的帧数可能小于动画的实际帧长度，表明该图层在缺失帧的画面中图像为静止，此时需要复制最后一个帧的内容，填充至缺失画面的帧中  

In [86]:
# 处理帧内容，主要是插帧
# 对于Delay > 1且Interpolated = true的关键帧，其中一些值如果相对于下一个关键帧发生了变化，需要均匀分布在中间帧
# 需要插值的值：Position、Scale、ColorTint、ColorOffset、Rotation
def Interpolate(FrameList,FrameNum):
    # 如果没有关键帧，就退出，要不然要出大问题
    if FrameList == []:
        return []
    InterFrameList = []
    for f in range(len(FrameList)):
        Frame = FrameList[f]
        InterFrameList.append(Frame)
        # 当Delay == 1的情况：直接跳过循环。顺便就可以把最后一个关键帧跳过了
        for d in range(Frame["Delay"]-1):
            if d == 0:
                break
            # 当1且Interpolated == false 的情况，循环赋值
            if Frame["Interpolated"] == "false":
                InterFrameList.append(Frame)
            else:
                FrameInsert = {}
                FrameNext = FrameList[f+1]
                for Key in Frame:
                    if(Key == "Visible")or(Key == "Interpolated"):
                        FrameInsert[Key] = Frame[Key]
                    elif(Frame[Key] == FrameNext[Key]):
                        FrameInsert[Key] = Frame[Key]
                    else:
                        if (Key == "XPosition") or (Key =="YPosition")or (Key =="XScale") or (Key =="YScale") or (Key =="RedTint") or (Key =="GreenTint") or (Key =="BlueTint") or (Key =="AlphaTint") or (Key =="RedOffset") or (Key =="GreenOffset") or (Key =="BlueOffset") or (Key =="Rotation"):
                            FrameInsert[Key] = int(Frame[Key]) + (int(FrameNext[Key])-int(Frame[Key]))/int(Frame['Delay'])
                        else:
                            FrameInsert[Key] = Frame[Key]
                InterFrameList.append(FrameInsert)

    while FrameNum > len(InterFrameList):
        InterFrameList.append(InterFrameList[-1])
    return InterFrameList



## 处理帧图片
根据读取图片，帧中的各属性值处理图片，得到图片列表  


In [87]:
# 读取并处理帧图片
import cv2
import numpy as np

def PreprocessImg(FrameList,path):
    if FrameList == []:
        return []
    ImgFrameList = []
    Img_BGRA = cv2.imread(path,cv2.IMREAD_UNCHANGED)# 读取BGR+alpha通道
    img_RGBA = cv2.cvtColor(Img_BGRA, cv2.COLOR_BGRA2RGBA)# BGRA转RGBA
    for Frame in FrameList:
        # 裁剪图片
        CropedImg = img_RGBA[ int(Frame['YCrop']) : (int(Frame['YCrop'])+int(Frame['Height'])) , int(Frame['XCrop']) : (int(Frame['XCrop'])+int(Frame['Width'])) ]
        # 拉伸变换
        NewWidth = int(int(Frame['XScale'])*int(Frame['Width'])/100)
        NewHeight =  int(int(Frame['YScale'])*int(Frame['Height'])/100)
        # !如果是负说明要发生翻转，然后再变为正数进行缩放，否则会报错
        if NewWidth < 0:
            CropedImg = cv2.flip(CropedImg, 1)
            NewWidth = -NewWidth
            Frame['XScale'] = -Frame['XScale']
        if NewHeight < 0:
            CropedImg = cv2.flip(CropedImg, 0)
            NewHeight = -NewHeight
            Frame['YScale'] = -Frame['YScale']
        NewSize =  (NewWidth,NewHeight)
        ResizedImg = cv2.resize(CropedImg,NewSize,interpolation=cv2.INTER_NEAREST)
        # 根据画布重新裁剪或扩充画面
        AddSizeList = []
        AddSizeList.append(46 + int(Frame["YPosition"]) - int(Frame["YPivot"]*Frame["YScale"]/100))
        AddSizeList.append(18 - int(Frame["YPosition"]) + int(Frame["YPivot"]*Frame["YScale"]/100) - int(Frame["Height"]*Frame["YScale"]/100))
        AddSizeList.append(32 + int(Frame["XPosition"]) - int(Frame["XPivot"]*Frame["XScale"]/100))
        AddSizeList.append(32 - int(Frame["XPosition"]) + int(Frame["XPivot"]*Frame["XScale"]/100) - int(Frame["Width"]*Frame["XScale"]/100))
        CropSizeList = []
        for i in range(len(AddSizeList)):
            AddSize = AddSizeList[i]
            if AddSize < 0:
                CropSizeList.append(int(-AddSize))
                AddSizeList[i] = 0
            else:
                CropSizeList.append(int(0))
        ImgSize = ResizedImg.shape
        preFullfillImg = ResizedImg[ CropSizeList[0] : ImgSize[0]-CropSizeList[1] , CropSizeList[2] : ImgSize[1]-CropSizeList[3] ]
        # 可能裁剪后图片的宽或者高为0，此时直接返回一张空白图片，否则继续执行会报错（好像裁剪后为负数会自动变为0）
        if preFullfillImg.size == 0:
            FullfillImg = np.zeros((400,400,4), np.uint8) # 不知道后面的格式具体内容
        else:
            FullfillImg = cv2.copyMakeBorder(preFullfillImg, AddSizeList[0], AddSizeList[1],  AddSizeList[2],  AddSizeList[3], borderType=cv2.BORDER_REPLICATE)
        ImgFrameList.append(FullfillImg)
    return ImgFrameList

## 合并图层  
一个动画中是由多个图层中的帧列表堆叠形成的，因此需要将不同图层的图片列表进行堆叠合并。  

In [88]:
# 合并图层
def MergeFrameImg(LayerList,LayerInfo,FrameNum):
    AnimationFrameImgList = []
    for Layer in LayerList:
        FrameList = Layer["FrameList"]
        ImgPath = "./gfx/" + LayerInfo[LayerList.index(Layer)]["Path"]
        img =  cv2.imread(ImgPath)
        FrameList = Interpolate(Layer["FrameList"],FrameNum)
        FrameImgList = PreprocessImg(FrameList,ImgPath)
        if AnimationFrameImgList == []:
            AnimationFrameImgList = FrameImgList
        else:
            for i in range(len(FrameImgList)):
                MergeImg = AnimationFrameImgList[i]
                FrameImg = FrameImgList[i]
                for x in range(FrameImg.shape[0]):
                    for y in range(FrameImg.shape[1]):
                        AlphaCannel = FrameImg[x,y][3]
                        if AlphaCannel != 0:
                            MergeImg[x,y] = FrameImg[x,y]
    return AnimationFrameImgList

## 合成GIF 动画
将png图片列表合成gif动画文件  
用了两种方法  
一个是imageio，还不知道怎么得到透明背景的gif动画  
一个是pic，能够得到透明背景的gif动画，但是需要将numpy格式的图片转化为pic.image格式。后续可能会尝试将前面的步骤改为用pic处理

In [89]:
# 合成gif动图
import imageio
import os
import sys
def png_gif_imageio(path,ImgList):
    png_lst = os.listdir(path)
    imageio.mimsave("result_imgeio.gif", ImgList, 'GIF', duration=1/12)

from PIL import Image
import os
def png_gif_pic(ImgList,FileName):
    PicImgList = []

    for CV2Img in ImgList: 
        PicImg = Image.fromarray(CV2Img) # numpy 转 image类
        PicImgList.append(PicImg)
    PicImgList[0].save("./result/" + FileName+".gif", save_all=True, append_images=PicImgList[1:],duration=18,transparency=0,loop=0,disposal=2)

## 主程序
调用以上方法，读取.anm2和.png文件，合成gif动画  
目前是直接指定的文件名，以及指定的动画文件中的动画。后续会设置一些参数，可以指定选择的动画文件，以及生成其中哪些动画

In [90]:
AnimationPath = "./gfx/001.000_player.anm2"
DOMTreeRoot = DecodeXML(AnimationPath)

AnimationInfo = GetAnimationInfo(DOMTreeRoot)
AnimaitionList = GetAnimationFrame(DOMTreeRoot)

# for Animation in AnimaitionList:
Animation = AnimaitionList[35]
AnimationName = Animation["Name"]
print(AnimationName)
LayerList = Animation["LayerAnimationList"]
LayerInfo = AnimationInfo["LayerInfo"]
FrameNum = Animation["FrameNum"]
AnimationFrameImgList = MergeFrameImg(LayerList,LayerInfo,FrameNum)

png_gif_pic(AnimationFrameImgList,AnimationName)

DeathTeleport


# 测试
测试

In [91]:
DOMTreeRoot = DecodeXML("./gfx/001.000_player.anm2")
AnimaitionList = GetAnimationFrame(DOMTreeRoot)

def test0(DOMTreeRoot):
    AnimationInfo = GetAnimationInfo(DOMTreeRoot)
    print(AnimationInfo)
    # for Key in AnimationInfo:
        # print(AnimationInfo[Key])

# 遍历所有关键帧
def test1(AnimaitionList):
    # print(AnimaitionList)
    for Animation in AnimaitionList:
        print(Animation["Name"])
        for Layer in Animation["LayerAnimationList"]:
            # print(Layer)
            for Frame in Layer["FrameList"]:
                print(Frame)
                for Key in Frame:
                    print(Frame[Key],end = " ")
                print("")

# 遍历所有帧（关键帧和插入帧）
def test2(AnimaitionList):
    for Animation in AnimaitionList:
        print(Animation["Name"])
        if Animation["Name"] == "Pickup":
            for Layer in Animation["LayerAnimationList"]:
                # print(LayerList)
                FrameList = Layer["FrameList"]
                InterFrameList = Interpolate(FrameList)
                # for Frame in InterFrameList:
                for f in range(len(InterFrameList)):
                    Frame = InterFrameList[f]
                    if f > 0:
                    # print(Layer)
                        for Key in Frame:
                            print(Frame[Key],end = " ")
                        print("")

# 遍历帧图片列表
def test3(AnimaitionList):
    Animation = AnimaitionList[0]
    LayerAnimationList = Animation["LayerAnimationList"]
    Layer = LayerAnimationList[12]
    FrameList = Layer["FrameList"]
    FrameList = Interpolate(FrameList)
    imgpath = "H:\\Steam\\steamapps\\common\\The Binding of Isaac Rebirth\\tools\IsaacAnimationEditor\\gfx\\characters\\costumes\\Character_001_Isaac.png"
    ImgList = PreprocessImg(FrameList,imgpath)


# test3(AnimaitionList)